In [ ]:
import json
import os
from collections import defaultdict

def preprocess_slangs_by_age_topic(json_dir, output_dir):
		# 출력 폴더 생성 (없으면 자동 생성)
    os.makedirs(output_dir, exist_ok=True)

    # 주제_연령_성별 조합별로 은어 데이터를 그룹화
    grouped_slangs = defaultdict(list)

    print(len(os.listdir(json_dir)))
    length = len(os.listdir(json_dir))
    count = 0
		
		# 입력 폴더 내 모든 JSON 파일 처리
    for file_name in os.listdir(json_dir):
        count +=1
        if not file_name.endswith(".json"):
            continue
            
        # JSON 파일 열기
        with open(os.path.join(json_dir, file_name), "r", encoding="utf-8") as f:
            data = json.load(f)
        # 데이터 추출: 주제, 연령대, 성별
        topic = data["SubCategory"]                 # 주제 (SNS, 경제 등)
        gender = data["Speakers"][0]["Gender"]      # 여성, 남성
        age_group = data["Speakers"][0]["AgeGroup"] # 연령대 (10대, 50대 등)
				
				# 정보가 부족할 경우 스킵
        if not topic or not age_group or not gender:
            continue
        
        # 추출 정보를 넣을 json 파일 이름
        key = f"{topic}_{age_group}_{gender}"
        
        # 은어 문장, 은어 문장의 표준어 풀이 추출
        for dialog in data.get("Dialogs", []):
            word_info = dialog.get("WordInfo", [])

            # 정보가 부족할 경우 스킵(WordInfo가 null이거나 리스트가 아닌 경우 해당 dialog 스킵)
            if word_info is None or not isinstance(word_info, list):
                continue

            slang_pair = {
                "slang_sentence":dialog["SpeakerText"],      # 은어 문장
                "standard_sentence":dialog["TextConvert"],    # 은어 문장의 표준어 풀이
                # 문장에 사용된 은어의 뜻풀이
                "slang_meaning_pairs":[
                    {
                        "slang": slang_meaning.get("Word", ""),
                        "meaning": slang_meaning.get("WordDefine", "")
                    } 
                    for slang_meaning in word_info
                ]
            }
            grouped_slangs[key].append(slang_pair) # 주제_연령_성별이 같은 은어를 같은 파일에 저장
        
        # 진행 상황 출력
        if(count%1000==0):
            print(f"{count}/{length}")
    print("모든 은어 데이터를 주제_연령_성별별로 분류 완료")

    # 그룹별 JSON 저장
    for key, slang_pair in grouped_slangs.items():
        with open(os.path.join(output_dir, f"{key}.json"), "w", encoding="utf-8") as f:
            json.dump(slang_pair, f, ensure_ascii=False, indent=2)
        print(f"{key} json파일 생성 완료")
    print("전체 은어 전처리 완료")

# ==================== 실행부분 ====================
# 상위 폴더 경로
root_folder = '은어 데이터 셋이 저장된 폴더 경로'
# 전처리 파일 저장할 폴더
output_folder = '전처리된 파일들이 저장될 폴더 경로'

# 출력 폴더가 없다면 생성
os.makedirs(output_folder, exist_ok=True)

# 전처리 실행
preprocess_slangs_by_age_topic(
            json_dir=root_folder, 
            output_dir=output_folder
        )
print("모든 파일 전처리가 완료되었습니다.")